In [3]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import nltk
import emoji
import imaplib
import email
from email.header import decode_header
import pandas as pd
from datetime import datetime, timedelta
import pytz
import configparser
import json

In [ ]:

config = configparser.ConfigParser()
config.read("app.config")


IMAP_SERVER = config["fetch"]["IMAP_SERVER"]
SMTP_SERVER = config["fetch"]["SMTP_SERVER"]
EMAIL_ACCOUNT = config["fetch"]["EMAIL_ACCOUNT"]
APP_PASSWORD = config["fetch"]["APP_PASSWORD"]

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, APP_PASSWORD)
mail.select('inbox')

since_date = (datetime.utcnow() - timedelta(hours=24)).strftime('%d-%b-%Y')
result, data = mail.uid('search', None, f'(SINCE {since_date})')

uids = data[0].split()


def decode_mime_words(s):
    if not s:
        return ""
    decoded = decode_header(s)
    return ''.join([
        part[0].decode(part[1] or 'utf-8') if isinstance(part[0], bytes) else str(part[0])
        for part in decoded
    ])

# Filter emails from the last 5 hours
emails = []
cutoff_time = datetime.utcnow().replace(tzinfo=pytz.UTC) - timedelta(hours=5)

for uid in uids:
    result, msg_data = mail.uid('fetch', uid, '(RFC822)')
    if result != 'OK':
        continue

    raw_email = msg_data[0][1]
    msg = email.message_from_bytes(raw_email)

    date_str = msg.get('Date')
    try:
        email_date = email.utils.parsedate_to_datetime(date_str)
        if email_date.tzinfo is None:
            email_date = email_date.replace(tzinfo=pytz.UTC)
    except:
        continue

    if email_date < cutoff_time:
        continue  

    subject = decode_mime_words(msg.get('Subject'))
    sender = decode_mime_words(msg.get('From'))
    recipient = decode_mime_words(msg.get('To'))

   
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain" and not part.get("Content-Disposition"):
                charset = part.get_content_charset()
                body = part.get_payload(decode=True).decode(charset or 'utf-8', errors='ignore')
                break
    else:
        charset = msg.get_content_charset()
        body = msg.get_payload(decode=True).decode(charset or 'utf-8', errors='ignore')

    emails.append({
        'uid': uid.decode(),
        'to': recipient,
        'from': sender,
        'subject': subject,
        'content': body,
        'timestamp': email_date.strftime('%Y-%m-%d %H:%M:%S %Z')
    })

mail.logout()


df = pd.DataFrame(emails)

df.head()


,uid,to,from,subject,content,timestamp
0,16458,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,Security alert,[image: Google]\r\nfetchemail was granted acce...,2025-04-10 10:21:16 UTC
1,16459,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,"Yash, plan your festive journey now! 🌼",Redbus Welcome\r\n\r\n \r\n\r\n\r\n\r\n \r\n\r...,2025-04-10 16:24:19 UTC+05:30
2,16460,yashjoshi30052002@gmail.com,Mail Delivery Subsystem <mailer-daemon@googlem...,Delivery Status Notification (Failure),\r\n** Address not found **\r\n\r\nYour messag...,2025-04-10 03:55:55 UTC-07:00
3,16461,yashjoshi30052002@gmail.com,Semrush Team <news@team.semrush.com>,Quick on-page SEO fixes you can do today,Here for higher rankings\r\nEver heard of on-p...,2025-04-10 06:24:45 UTC-05:00
4,16462,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,2-Step Verification turned on,[image: Google]\r\n2-Step Verification turned ...,2025-04-10 11:48:36 UTC


In [4]:

def clean_content(text):
    text = str(text)  
    text = text.replace("\r\n", " ")  
    text = re.sub(r'\s+', ' ', text) 
    return text.strip() 

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub('', text)


def remove_punc(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    text = str(text) 
    new_text = []
    for word in text.split():
        if word.lower() in stopwords.words('english'):
            new_text.append(' ')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return ' '.join(x)



df['content'] = df['content'].str.lower()
df['content'] = df['content'].apply(clean_content)
df['content'] = df['content'].apply(remove_url)
df['content'] = df['content'].apply(remove_punc)
for i in range (0 ,df.shape[0]) :
    df['content'][i] = remove_stopwords(df['content'][i])
df['content'] = df['content'].apply(lambda x : emoji.demojize(x))



df['subject'] = df['subject'].str.lower()
df['subject'] = df['subject'].apply(clean_content)
df['subject'] = df['subject'].apply(remove_url)
df['subject'] = df['subject'].apply(remove_punc)
for i in range (0 ,df.shape[0]) :
    df['subject'][i] = remove_stopwords(df['subject'][i])

df['subject'] = df['subject'].apply(lambda x : emoji.demojize(x))




C:\Users\DELL\AppData\Local\Temp\ipykernel_8372\198205819.py:34: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['content'][i] = remove_stopwords(df['content'][i])
C:\Users\DELL\AppData\Local\Temp\ipykernel_8372\198205819.py:34: FutureWarni

In [5]:
from datetime import datetime

df.to_pickle("emails.pkl")
now = datetime.now()
timestamp_str = now.strftime("%Y%m%d_%H%M%S")
filename = f"emails_{timestamp_str}.pkl"
df.to_pickle(filename)

print(f"DataFrame saved to: {filename}")

DataFrame saved to: emails_20250410_193340.pkl


In [6]:
df.head()

,uid,to,from,subject,content,timestamp
0,16458,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,security alert,image google fetchemail granted access g...,2025-04-10 10:21:16 UTC
1,16459,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,yash plan festive journey :blossom:,redbus welcome plan bus journey new year...,2025-04-10 16:24:19 UTC+05:30
2,16460,yashjoshi30052002@gmail.com,Mail Delivery Subsystem <mailer-daemon@googlem...,delivery status notification failure,address found message wasnt delivered re...,2025-04-10 03:55:55 UTC-07:00
3,16461,yashjoshi30052002@gmail.com,Semrush Team <news@team.semrush.com>,quick onpage seo fixes today,higher rankings ever heard onpage seo it...,2025-04-10 06:24:45 UTC-05:00
4,16462,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,2step verification turned,image google 2step verification turned yashj...,2025-04-10 11:48:36 UTC
